# $target secondary eclipse analysis

This notebook template is made for MuSCAT2 secondary eclipse analyses and uses the reduced light curves saved in *fits* files.

In [ ]:
%pylab inline
rc('figure', figsize=(13,6))

In [ ]:
from muscat2ta import M2EclipseLPF

## Initialise the log posterior function

The secondary eclipse analysis is done with a specialised `M2EclipseLPF` log posterior function. By default, the class reads all the `fits` light curves saved in the `results` directory, doesn't downsample the data, and uses a GP systematics model.

#### M2EclipseLPF initialisation parameters

- ``name:`` the name of the *analysis* that will also be used when saving the results. This doesn't need to equal to the name of the target.

- ``datadir:`` directory where the reduced MuSCAT2 light curves are stored.

- ``pattern:`` a glob pattern used to select the data files from the data directory, defaults to ``'*.fits'``

- ``model_baseline:`` use a linear baseline model if set to `True`. Not necessary since we're using a GP systematics model.

- ``downsample:`` minimum exposure time in seconds. We aren't downsampling (binning) the light curve by default (this is often done already in the data reduction phase). However, downsampling can be enabled by setting this to a ``float`` value giving the desired average exposure time in seconds.


In [ ]:
lpf = M2EclipseLPF('$target', downsample=None)

## Set the priors

Transit centre (`tc`), orbital period (`p`), planet-star area ratio (`k2`), stellar density (`rho`), and impact parameter (`b`) should be given informative priors for a secondary eclipse analysis. The transit centre and impact parameter correspond to the values for the primary transit (the eclipse centre and impact parameter depend also on the eccentricity and argument of periastron), and the area ratio is simpy radius ratio squared ($k^2$).

**Setting priors:**
The priors can be set using the `lpf.set_prior` method that takes the parameter name, prior, and prior parameters as its arguments. The prior can be either `"UP"` for uniform prior, `"NP"` for normal prior, or any object that implements `logpdf(x)` method to calculate the log prior density at point `x`. For a normal prior, the two parameters define the prior mean and standard deviation, and for a uniform prior they define the prior start and end.

For example, a normal prior on stellar density (`rho`) can be set as

    lpf.set_prior('rho',  'NP', 2.15, 0.10)
    
while a uniform prior on impact parameter can be set as

    lpf.set_prior('rho',  'UP', 1.2, 3.2)


**Eccentricity:** The eccentricity and $\omega$ are parametrised using $\text{secw} = \sqrt{e} \cos \omega$ and $\text{sesw} = \sqrt{e} \sin \omega$. These parameters have tight zero-centered priors by default that force a
circular orbit. However, this can be overridden by setting the priors on `secw` and `sesw`.

**Necessary parameters:** these parameters need good informative priors for the analysis to make any sense whatsoever. Plese replace *TC*, *P*, and *K2* with the transit centre, orbital period, and area ratio estimates, and *TCE*, *PE*, and *K2E* with their uncertainties.

In [ ]:
lpf.set_prior('tc',  'NP', TC, TCE)
lpf.set_prior('p',   'NP', P, PE)
lpf.set_prior('k2',  'NP', K2, K2E)

**Very recommended parameters:** while not absolutely necessary, these priors should be set to constrain the shape and duration of the signal that we expect. Please uncomment these lines and replace *RHO* and *B* with the stellar density (in g/cm$^3$) and impact parameter estimates, and *RHOE* and *BE* with their uncertainties.

In [ ]:
#lpf.set_prior('rho', 'NP', RHO, RHOE)
#lpf.set_prior('b',   'NP', B, BE)

### Print the parametrisation and priors

The parametrisation and priors can be shown by printing `lpf.ps`. This isn't absolutely necessary, but can be useful to see the number of free parameters and that the priors are all what they're supposed to be.

In [ ]:
lpf.ps

## Fit the model

Fit the model using a global optimiser. Here the most important thing is that the number of parameter vectors (`npop`) is at least twice as large as the number of free model parameters (shown above). As with the data reduction, this step can be repeated until convergence.

In [ ]:
lpf.optimize_global(200, npop=200)

In [ ]:
lpf.plot_light_curves();

## Sample the posterior

Create a sample from the posterior using MCMC. In most cases you should run the MCMC for at least 10000 samples, but this can be done in small batches to see how the posteriors evolve. 

In [ ]:
lpf.sample_mcmc(500, repeats=3)

In [ ]:
lpf.plot_flux_ratio_posteriors(lpf, figsize=(13,3));

## Save the results

Save the results for visualisation and further analysis. It's a good idea to keep the modelling separate from the rest of the analysis.

In [ ]:
lpf.save()

---

<center> MuSCAT2 analysis pipeline </center>